In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import multiprocessing as mp
from multiprocessing import cpu_count
from selenium import webdriver
from functools import partial
from datetime import datetime
import time
import tqdm
import utils
import traceback
import pandas as pd

from webdriver import get_web_driver
tdy = utils.get_today()
start_time = time.time()
SITEMAP_URL = "https://www.openrice.com/en/hongkong/restaurants"

start_time = time.time()
print("TODAY =", tdy)
print("CPU COUNT: " + str(cpu_count()))

avail_core_num = cpu_count()-1

TODAY = 2020-11-15 22:02:49
TODAY = 2020-11-15 22:02:49
CPU COUNT: 12


In [2]:
option = webdriver.ChromeOptions()
driver = webdriver.Chrome('/Users/ziwon/executables/chromedriver86', options=option)
driver.get(SITEMAP_URL)
# driver.maximize_window()
driver.implicitly_wait(3)

wait = WebDriverWait(driver, 10)

# Tick "Opens 24 Hours" & "Open Till Late"

In [3]:
# List all Features when page loads
features_filter = wait.until(ec.visibility_of_element_located((By.ID, "locationFilter")))
more_btn = features_filter.find_element_by_xpath("./div[@class='more-btn js-more-btn']")
more_btn.click()

In [4]:
sai_kung = wait.until(ec.element_to_be_clickable((By.CSS_SELECTOR, 'div[data-tag="Sai Kung"]')))
if 'active' not in sai_kung.get_attribute('class'):
    sai_kung.find_element_by_css_selector('.icon-checkbox').click()

In [5]:
hang_hau = wait.until(ec.element_to_be_clickable((By.CSS_SELECTOR, 'div[data-tag="Hang Hau"]')))    
if 'active' not in hang_hau.get_attribute('class'):
    hang_hau.find_element_by_css_selector('.icon-checkbox').click()

In [6]:
cont = wait.until(ec.element_to_be_clickable((By.CSS_SELECTOR, 'span[class="btn btn-block btn-filter js-btn-confirm"]')))
cont.click()

# Get list of restaurants incl. (NAME, REGION, RATING, REVIEW)

Get URLs of all restaurants

In [7]:
cols = ['District', 'Name', 'URL', 'Address', 'Price_Range', 'Cuisine', 'Review_Count', 'Happy_Ct', 'Sad_Ct', 'Bookmark_Ct']
df_restaurants = pd.DataFrame(columns=cols)

In [10]:
len(df_restaurants)

250

In [9]:
results = []
has_next = True
has_next_wait = WebDriverWait(driver, 5)
while has_next:
    try:
        next_page_button = has_next_wait.until(ec.element_to_be_clickable((By.CSS_SELECTOR, "a.pagination-button.next")))
    except:
        print("END OF PAGES REACHED. NO MORE TO PARSE.")
        has_next = False

    # Parse all restaurants in current page then move on
    restaurants = driver.find_elements_by_css_selector('li.sr1-listing-content-cell.pois-restaurant-list-cell')

    for restaurant in restaurants:
        name = restaurant.find_element_by_xpath('./div/section/div/h2/a[@href]').text
        print(name)
        url = restaurant.find_element_by_xpath('./div/section/div/h2/a[@href]').get_attribute('href')
        num_bookmarks = restaurant.find_element_by_xpath('./div/section/section[@class="js-openrice-bookmark openrice-bookmark"]').text
        num_bookmarks = int(''.join(filter(str.isdigit, num_bookmarks)))
        num_reviews = restaurant.find_element_by_xpath('./div/section/div[@class="subtitle-wrapper pois-restaurant-list-cell-top-info"]').text
        num_reviews = int(''.join(filter(str.isdigit, num_reviews)))
        try:
            address = restaurant.find_element_by_xpath('./div/section/div[@class="details-wrapper"]/div[@class="central-content-container"]/div/div[@class="icon-info address"]/span').text
            district = restaurant.find_element_by_xpath('./div/section/div[@class="details-wrapper"]/div[@class="central-content-container"]/div/div[@class="icon-info address"]/span/a').text
        except:
            print("Delivery Only")
            address = "Delivery Only"
            district = "Delivery Only"

        price_range = restaurant.find_element_by_xpath('./div/section/div[@class="details-wrapper"]/div[@class="central-content-container"]/div/div[@class="icon-info"]/div/div[@class="icon-info icon-info-food-price"]/span').text
        cuisine = restaurant.find_element_by_xpath('./div/section/div[@class="details-wrapper"]/div[@class="central-content-container"]/div/div[@class="icon-info"]/div/div[@class="icon-info icon-info-food-name"]/ul').text

        happy_ct = restaurant.find_element_by_xpath('./div/section/div[@class="details-wrapper"]/div[@class="right-content-container pois-restaurant-list-cell-content-right-info-rating "]/div/span[@class="score score-big highlight"]').text
        sad_ct = restaurant.find_element_by_xpath('./div/section/div[@class="details-wrapper"]/div[@class="right-content-container pois-restaurant-list-cell-content-right-info-rating "]/div/span[@class="score highlight"]').text

        df_restaurants.loc[len(df_restaurants)] = {
            'District': str(district),
            'Name': str(name),
            'URL': str(url),
            'Address': str(address),
            'Price_Range': str(price_range),
            'Cuisine': str(cuisine),
            'Review_Count': str(num_reviews),
            'Happy_Ct': str(happy_ct),
            'Sad_Ct': str(sad_ct),
            'Bookmark_Ct': str(num_bookmarks)
        }
    next_page_button.location_once_scrolled_into_view
    next_page_button.click()

WA-Theater Restaurant 食之劇場餐廳
Little Cove Espresso
Shin Mapo BBQ (East Point City) 新麻蒲 BBQ (東港城)
Thai Dao 泰道
The Alchemist Café (Hong Kong Velodrome) 牧羊少年咖啡館 (香港單車館)
Mee...lemongrass
let's Jam 一起果醬
Maguro Sekaiichiba 鮪世市場
Casserole Buddies (East Point City) 砂煲兄弟 (東港城)
Sai Kung Cafe & Bakery 西貢咖啡餅店
Yin At Home 隱居
Golden Chicken 金雞去骨海南雞專門店
Padstow Restaurant & Bar
Beach BBQ King 西貢海灣會燒烤會所
MAXIM’S AND YOU 美心小品
Beans Brekkie Room
NeNe Chicken (東港城)
Gastrobox by Okapi Cafe
Oreryu Shio Ramen (East Point City) 俺流塩らーめん (東港城)
Momentai 無問題
Bakso Flagship Store 白鬚旗艦店
Hushush Ice Cream Coffee
Itamama (東港城)
Umimachidon (East Point City) 海街丼 (東港城)
Mr. Steak (East Point City) 牛扒先生 (東港城)
Cow Bean (TKO Gateway)
清心齋
Cafe Alley
Bijin Nabe 美人鍋
Orca
Chuen Kee Seafood Restaurant 全記海鮮菜館
Sing Kee Seafood Restaurant 勝記海鮮酒家
Blacksmith Coffee Shop
E.Tea 茶星人
Momoku (TKO Gateway) 韓樂 (TKO Gateway)
髀魔-雞髀第一店 (福民路)
Swiss Cafe 瑞士咖啡室
Hazukido 八月堂
TeaWood Taiwanese Cafe & Restaurant 茶木‧台式休閒餐廳
Pho Le 錦麗
Shiba Taro Cafe 柴太郎

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=86.0.4240.193)


### Get all restaurants in page first, until ">>" Button not shown anymore

In [ ]:
cols = ['District', 'Name', 'Opening_Times', 'Address', 'Score', 'Bookmarks', 'Price', 'Categories', 'URL']
df_restaurants = pd.DataFrame(cols) #creates a new dataframe that's empty

def parse_openrice_restaurant(url):
    rstr_driver = get_web_driver()
    rstr_driver.get(url)
    rstr_wait = WebDriverWait(rstr_driver, 10)
    
    center_area = rstr_wait.until(ec.visibility_of_element_located((By.CSS_SELECTOR, "div.center-area")))
    left_col = rstr_wait.until(ec.visibility_of_element_located((By.CSS_SELECTOR, "div.left-col")))    
    
    if (len(center_area.find_elements_by_css_selector(".poi-with-other-status")) != 0):
            poi_other_status = center_area.find_element_by_css_selector(".poi-with-other-status").text
            if poi_other_status == '(MOVED)':
                new_link = center_area.find_element_by_css_selector(".new-location-link").get_attribute("href")
                parse_openrice_restaurant(new_link)
    else:
        try:
            name = center_area.find_element_by_css_selector(".name").text

            print("Parsing URL: " + url + " | Name: "+ name)

            try:
                name_smaller = center_area.find_element_by_css_selector(".smaller-font-name").text
            except:
                name_smaller = None
            try:
                header_score = center_area.find_element_by_css_selector(".header-score").text
            except: 
                header_score = None
            try:
                bookmark_count = center_area.find_element_by_css_selector(".header-bookmark-count.js-header-bookmark-count").text
            except: 
                bookmark_count = None
            try: 
                district = center_area.find_element_by_css_selector(".header-poi-district.dot-separator").text
            except: 
                district = None
            try:
                price_range = center_area.find_element_by_css_selector(".header-poi-price.dot-separator").text
            except: 
                price_range = None
            try:
                categories = center_area.find_element_by_css_selector(".header-poi-categories.dot-separator").text
            except:
                categories = None
            try:
                address = left_col.find_element_by_css_selector('.address-section').find_element_by_css_selector('.content').text
            except:
                address = None
    #         except Exception as e:
    #             print(e)
    #             print("Error - Cannot parse certain field. Manually Fill later.")
    #             name_smaller = None
    #             header_score = None
    #             bookmark_count = None
    #             district = None
    #             price_range = None
    #             categories = None
    #             address = None

            # Opening Hours
            try:
                opening_hours_section = left_col.find_element_by_css_selector('div.opening-hours-section.js-normal-and-special-opening-hours-section')
                opening_hours_days = opening_hours_section.find_elements_by_css_selector(".opening-hours-day")

                opening_date_times = {}
                for opening_hour_day in opening_hours_days:
                    date = opening_hour_day.find_element_by_css_selector('.opening-hours-date').text
                    time = opening_hour_day.find_element_by_css_selector('.opening-hours-time').text
                    opening_date_times[date] = time

                print("opening_date_times: " + str(opening_date_times))
            except Exception as e:
                print(e)
    #             print(traceback.format_exc())
                print("ERROR - Some error occurred while parsing Opening Date Times for: " + url)
                print("Fill in manually later")
                opening_date_times = []

            dic = {
                'District': str(district),
                'Name': str(name),
                'Opening_Times': str(opening_date_times),
                'Address': str(address),
                'Score': str(header_score),
                'Bookmarks': str(bookmark_count),
                'Price': str(price_range),
                'Categories': str(categories),
                'URL': str(url)
            }

            df_restaurants.loc[len(df_restaurants)] = dic
            print("Added " + name + " to DF_Restaurants.")
        except Exception as e:
            print(e)
            print(traceback.format_exc())
            print("----- ERROR -----")
            rstr_driver.close()
            rstr_driver.quit()
            return None
    
    
    rstr_driver.close()
    rstr_driver.quit()   

In [ ]:
results = []
has_next = True
has_next_wait = WebDriverWait(driver, 5)
while has_next:
    try:
        next_page_button = has_next_wait.until(ec.visibility_of_element_located((By.CSS_SELECTOR, "a.pagination-button.next")))
    except:
        print("END OF PAGES REACHED. NO MORE TO PARSE.")
        has_next = False
    
    # Parse all restaurants in current page then move on
    restaurants = driver.find_elements_by_css_selector('li.sr1-listing-content-cell.pois-restaurant-list-cell')
    
    for restaurant in restaurants:
        url = restaurant.find_element_by_xpath('./div/section/div/h2/a[@href]').get_attribute('href')
        result = parse_openrice_restaurant(url)
        results.append(result)
    
    next_page_button.click()

# SAVE RESULTS

In [14]:
results=df_restaurants

In [18]:
results.to_csv('restaurant_info.tsv', sep = '\t')


In [16]:
len(results)

250

In [ ]:
ex = str(ex).replace('(', '').replace(')', '')

In [ ]:
ex = ex.replace('[[', "'[[").replace(']]', "]]'")

In [ ]:
str(ex)

In [ ]:
re.sub(r"(?!(([^']*'){2})*[^']*$),", '', ex)

In [ ]:
import re
with open('results_2.txt', 'w+') as f:
    for item in results:
        removed_par = str(item).replace('(', '').replace(')', '')
        remove_array_comm = removed_par.replace('[[', "'[[").replace(']]', "]]'")
        removed_quotes = re.sub(r"(?!(([^']*'){2})*[^']*$),", '', remove_array_comm)
        f.write(removed_quotes + '\n')

In [ ]:
# restaurants = driver.find_elements_by_css_selector('li.sr1-listing-content-cell.pois-restaurant-list-cell')
# restaurant = restaurants[0]
# url = restaurant.find_element_by_xpath('./div/section/div/h2/a[@href]').get_attribute('href')
